In [ ]:
!pip install xgboost

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier,XGBRegressor
from sklearn.preprocessing import StandardScaler

In [ ]:
#lb_make = LabelEncoder()
#df = pd.read_csv('data/Accident_train.csv')
#df['Policing_Area'] = lb_make.fit_transform(df['Policing_Area'].astype(str))
#df['Weekday_of_Collision'] = lb_make.fit_transform(df['Weekday_of_Collision'].astype(str))
#if df['Collision_Severity'].values[0] == 'Predict':
    #print(df['Collision_Severity'].values[0])

In [ ]:
#def increaseCountSkewedClasses(df,values):
    #for i in values:
        #is_Collision_Severity = df['Collision_Severity'] == i
        #df_try = df[is_Collision_Severity]
        #print('previous length: ',len(df_try))
        #df = df.append([df_try]*4,ignore_index=True)
        #is_Collision_Severity = df['Collision_Severity'] == i
        #df_try = df[is_Collision_Severity]
        #print('new length: ',len(df_try))
    #return df

In [2]:
def readingData(Location):
    df = pd.read_csv(Location)
    #print('Shape Before incresing the count: ',df.shape)
    values = [1,2]
    #df = increaseCountSkewedClasses(df,values)
    #print('Shape After incresing the count: ',df.shape)
    lb_make = LabelEncoder()
    df['Policing_Area'] = lb_make.fit_transform(df['Policing_Area'].astype(str))
    df['Weekday_of_Collision'] = lb_make.fit_transform(df['Weekday_of_Collision'].astype(str))
    #Run only once
    if 'Collision_Severity' in df.columns:
        if df['Collision_Severity'].all() == 'Predict':
            #print(df['Collision_Severity'].values[0])
            del df['Collision_Severity']
        else:
            df['Collision_Severity_labels'] = df['Collision_Severity']
            del df['Collision_Severity']
    if 'Collision_Ref_No' in df.columns:
        del df['Collision_Ref_No']
    for n in df:
        mean = df[n].mode()
        df[n].fillna((df[n].mode()), inplace=True)
    #df.isna()
    if 'Collision_Severity_labels' in df.columns:
        #print('Akshay')
        X_TRAIN = df.iloc[:,0:-1]
        X_LABELS = df.iloc[:,-1:]
    else:
        X_TRAIN = df.iloc[:,0:]
        X_LABELS = pd.DataFrame(np.zeros((len(df), 1)))
    return X_TRAIN.values,X_LABELS.values

In [3]:
TRAIN,LABELS = readingData('data/Accident_train.csv')
#TEST,TEST_LABELS = readingData('data/Accident_train_head.csv')
TEST,_ = readingData('data/Accident_test.csv')

In [4]:
TEST[0]

array([20.,  1., 17.,  2., 21., 13., 60., 10.,  7.,  1.,  1.,  5.,  9.,
        6.,  1.])

In [5]:
TRAIN[0]

array([19.,  1.,  4.,  8., 14., 13., 60.,  1.,  1.,  1.,  1.,  2.,  9.,
        9.,  1.])

In [6]:
def splitDataSet(TRAIN,LABELS):
    x_train, x_test, y_train, y_test = train_test_split(TRAIN, LABELS, test_size=0.33, shuffle=True)
    #X, y = make_classification(n_samples=1000, n_features=4,n_informative=2, n_redundant=0,random_state=0, shuffle=True)
    return x_train, x_test, y_train, y_test

In [7]:
x_train, x_test, y_train, y_test = splitDataSet(TRAIN,LABELS)

In [ ]:
def LinearSVC(x_train,x_test,y_train,y_test=0):
    lin_clf = svm.LinearSVC()
    lin_clf.fit(x_train, y_train)
    prediction = lin_clf.predict(x_test)
    accuracy_score = accuracy_score(y_test,prediction)
    return prediction,accuracy_score

In [ ]:
def OVOsVC(x_train,x_test,y_train,y_test=0):
    clf = svm.SVC(decision_function_shape='ovo')
    clf.fit(x_train, y_train)
    prediction = clf.predict(x_test)
    #for i,n in enumerate(prediction):
        #if n != y_test[i]:
            #print(str(i)+' '+str(y_test[i]))
    accuracy_score = accuracy_score(y_test,prediction)
    return prediction,accuracy_score

In [ ]:
def DecisionTree(x_train,x_test,y_train,y_test=0):
    clf = DecisionTreeClassifier(criterion='entropy',min_impurity_decrease=1.0)
    clf.fit(x_train,y_train)
    prediction = clf.predict(x_test)
    accuracy_score = clf.score(x_test,y_test)
    return prediction,accuracy_score

In [ ]:
def kNN(x_train,x_test,y_train,y_test=0):
    neigh = KNeighborsClassifier(n_neighbors=2)
    neigh.fit(x_train,y_train) 
    KNeighborsClassifier(...)
    prediction = neigh.predict(x_test)
    accuracy_score = neigh.score(x_test,y_test)
    return prediction,accuracy_score

In [ ]:
def randomForest(x_train,x_test,y_train,y_test=0):
    clf = RandomForestClassifier(max_depth=9, random_state=3)
    sm = SMOTE(random_state=42)
    X_res, y_res = sm.fit_sample(x_train,y_train)
    clf.fit(X_res,y_res)
    prediction = clf.predict(x_test)
    accuracy_score = clf.score(x_test,y_test)
    return prediction,accuracy_score

In [23]:
def xgBoost(x_train,x_test,y_train,y_test=0):
    # fit model no training data#
    model = XGBClassifier()
    model.fit(x_train, y_train.ravel())
    # make predictions for test data
    predictions = model.predict(x_test)
    #accuracy_scores = accuracy_score(y_test.ravel(), predictions)
    return predictions

In [24]:
xgBoost(x_train, x_test, y_train, y_test)

D:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([3, 3, 3, ..., 3, 3, 3], dtype=int64)

In [25]:
np.savetxt("xgBoostsubmission.csv",xgBoost(TRAIN,TEST,LABELS),fmt='%.4e',header='Collision_Severity')

D:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
#np.savetxt("LinearSVCsubmission7.csv",LinearSVC(TRAIN,TEST,LABELS),fmt='%.4e',header='Collision_Severity')
#np.savetxt("OVOsVCsubmission7.csv",OVOsVC(TRAIN,TEST,LABELS),fmt='%.4e',header='Collision_Severity')
#np.savetxt("DecisonTreesubmission7.csv",DecisionTree(TRAIN,TEST,LABELS),fmt='%.4e',header='Collision_Severity')
#np.savetxt("RandomForestsubmission7.csv",randomForest(TRAIN,TEST,LABELS),fmt='%.4e',header='Collision_Severity')
#np.savetxt("xgBoostsubmission.csv",xgBoost(TRAIN,TEST,LABELS),fmt='%.4e',header='Collision_Severity')
#OVOsVC(TRAIN,TEST,LABELS,TEST_LABELS)